### 3. RuT5

In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict

TRAIN_FILE = "in_domain_train.csv"
IN_DOMAIN_DEV_FILE = "in_domain_dev.csv"
OUT_OF_DOMAIN_DEV_FILE = "out_of_domain_dev.csv"
TEST_FILE = "test.csv"


c:\Users\MVKiselev\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_splits(*, as_datasets):
    train_df, in_domain_dev_df, out_of_domain_dev_df, test_df = map(
        pd.read_csv, (TRAIN_FILE, IN_DOMAIN_DEV_FILE, OUT_OF_DOMAIN_DEV_FILE, TEST_FILE)
    )

    # concatenate datasets to get aggregate metrics
    dev_df = pd.concat((in_domain_dev_df, out_of_domain_dev_df))

    if as_datasets:
        train, dev, test = map(Dataset.from_pandas, (train_df, dev_df, test_df))
        return DatasetDict(train=train, dev=dev, test=test)
    else:
        return train_df, dev_df, test_df

In [3]:
import os
from argparse import ArgumentParser
from functools import partial
from shutil import rmtree

import numpy as np
from datasets import load_metric
from razdel import tokenize
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    T5Tokenizer,
    T5ForConditionalGeneration,
)

In [4]:
ACCURACY = load_metric("accuracy", keep_in_memory=True, trust_remote_code=True)
MCC = load_metric("matthews_correlation", keep_in_memory=True, trust_remote_code=True)

C:\Users\MVKiselev\AppData\Local\Temp\ipykernel_26808\1872012557.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  ACCURACY = load_metric("accuracy", keep_in_memory=True, trust_remote_code=True)


In [5]:
N_SEEDS = 10
N_EPOCHS = 20
LR_VALUES = (1e-4, 1e-3)
DECAY_VALUES = (0, 1e-4)
BATCH_SIZES = (128,)

POS_LABEL = "yes"
NEG_LABEL = "no"

In [6]:
def compute_metrics(p, tokenizer):
    string_preds = tokenizer.batch_decode(p.predictions, skip_special_tokens=True)
    int_preds = [1 if prediction == POS_LABEL else 0 for prediction in string_preds]

    labels = np.where(p.label_ids != -100, p.label_ids, tokenizer.pad_token_id)
    string_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    int_labels = []

    for string_label in string_labels:
        if string_label == POS_LABEL:
            int_labels.append(1)
        elif string_label == NEG_LABEL or string_label == "":  # second case accounts for test data
            int_labels.append(0)
        else:
            raise ValueError()

    acc_result = ACCURACY.compute(predictions=int_preds, references=int_labels)
    mcc_result = MCC.compute(predictions=int_preds, references=int_labels)

    result = {"accuracy": acc_result["accuracy"], "mcc": mcc_result["matthews_correlation"]}

    return result

In [7]:
def preprocess_examples(examples, tokenizer):
    result = tokenizer(examples["sentence"], padding=False)

    if "acceptable" in examples:
        label_sequences = []
        for label in examples["acceptable"]:
            if label == 1:
                target_sequence = POS_LABEL
            elif label == 0:
                target_sequence = NEG_LABEL
            else:
                raise ValueError("Unknown class label")
            label_sequences.append(target_sequence)

    else:
        # a hack to avoid the "You have to specify either decoder_input_ids or decoder_inputs_embeds" error
        # for test data
        label_sequences = ["" for _ in examples["sentence"]]

    result["labels"] = tokenizer(label_sequences, padding=False)["input_ids"]
    result["length"] = [len(list(tokenize(sentence))) for sentence in examples["sentence"]]
    return result

In [8]:
def main():
    tokenizer = T5Tokenizer.from_pretrained("sberbank-ai/ruT5-base")

    splits = read_splits(as_datasets=True)

    tokenized_splits = splits.map(
        partial(preprocess_examples, tokenizer=tokenizer),
        batched=True,
        remove_columns=["sentence"],
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8)

    # seed, lr, wd, bs
    dev_metrics_per_run = np.empty((N_SEEDS, len(LR_VALUES), len(DECAY_VALUES), len(BATCH_SIZES), 2))

    for i, learning_rate in enumerate(LR_VALUES):
        for j, weight_decay in enumerate(DECAY_VALUES):
            for k, batch_size in enumerate(BATCH_SIZES):
                for seed in range(N_SEEDS):
                    model = T5ForConditionalGeneration.from_pretrained("sberbank-ai/ruT5-base")

                    run_base_dir = f"sberbank-ai_ruT5-base_{learning_rate}_{weight_decay}_{batch_size}"

                    training_args = Seq2SeqTrainingArguments(
                        output_dir=f"checkpoints/{run_base_dir}",
                        overwrite_output_dir=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=learning_rate,
                        weight_decay=weight_decay,
                        num_train_epochs=N_EPOCHS,
                        lr_scheduler_type="constant",
                        save_strategy="epoch",
                        save_total_limit=1,
                        seed=seed,
                        fp16=True,
                        dataloader_num_workers=4,
                        group_by_length=True,
                        report_to="none",
                        load_best_model_at_end=True,
                        metric_for_best_model="eval_mcc",
                        optim="adafactor",
                        predict_with_generate=True,
                    )

                    trainer = Seq2SeqTrainer(
                        model=model,
                        args=training_args,
                        train_dataset=tokenized_splits["train"],
                        eval_dataset=tokenized_splits["dev"],
                        compute_metrics=partial(compute_metrics, tokenizer=tokenizer),
                        tokenizer=tokenizer,
                        data_collator=data_collator,
                    )

                    train_result = trainer.train()
                    print(f"{run_base_dir}_{seed}")
                    print("train", train_result.metrics)

                    os.makedirs(f"results/{run_base_dir}_{seed}", exist_ok=True)

                    dev_predictions = trainer.predict(
                        test_dataset=tokenized_splits["dev"], metric_key_prefix="test", max_length=10
                    )
                    print("dev", dev_predictions.metrics)
                    dev_metrics_per_run[seed, i, j, k] = (
                        dev_predictions.metrics["test_accuracy"],
                        dev_predictions.metrics["test_mcc"],
                    )

                    predictions = trainer.predict(test_dataset=tokenized_splits["test"], max_length=10)

                    string_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

                    int_preds = [1 if prediction == POS_LABEL else 0 for prediction in string_preds]
                    int_preds = np.asarray(int_preds)

                    np.save(f"results/{run_base_dir}_{seed}/preds.npy", int_preds)

                    rmtree(f"checkpoints/{run_base_dir}")

    os.makedirs("results_agg", exist_ok=True)
    np.save(f"results_agg/sberbank-ai_ruT5-base_dev.npy", dev_metrics_per_run)

In [9]:
main()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map: 100%|██████████| 2789/2789 [00:00<00:00, 4775.61 examples/s]
c:\Users\MVKiselev\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  5%|▌         | 62/1240 [14:01<4:35:56, 14.05s/it]c:\Users\MVKiselev\AppData\Local\Progra

{'eval_loss': 0.33953073620796204, 'eval_accuracy': 0.6752780767850736, 'eval_mcc': 0.0, 'eval_runtime': 283.9748, 'eval_samples_per_second': 9.814, 'eval_steps_per_second': 0.077, 'epoch': 1.0}


 10%|█         | 124/1240 [31:58<3:44:50, 12.09s/it] c:\Users\MVKiselev\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                    
 10%|█         | 124/1240 [36:42<3:44:50, 12.09s/it]

{'eval_loss': 0.27848392724990845, 'eval_accuracy': 0.6752780767850736, 'eval_mcc': 0.0, 'eval_runtime': 284.2194, 'eval_samples_per_second': 9.806, 'eval_steps_per_second': 0.077, 'epoch': 2.0}


 15%|█▌        | 186/1240 [48:20<2:37:42,  8.98s/it]  c:\Users\MVKiselev\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                    
 15%|█▌        | 186/1240 [51:03<2:37:42,  8.98s/it]

...

 40%|████      | 500/1240 [1:40:23<6:27:24, 31.41s/it] 

{'loss': 0.4248, 'grad_norm': 1.5460054874420166, 'learning_rate': 0.0001, 'epoch': 8.06}


 45%|████▌     | 558/1240 [1:48:42<1:56:10, 10.22s/it]c:\Users\MVKiselev\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

 45%|████▌     | 558/1240 [1:52:05<1:56:10, 10.22s/it]

{'eval_loss': 0.3030848801136017, 'eval_accuracy': 0.7186939361320416, 'eval_mcc': 0.28756234541207065, 'eval_runtime': 203.076, 'eval_samples_per_second': 13.724, 'eval_steps_per_second': 0.108, 'epoch': 9.0}


 48%|████▊     | 590/1240 [1:59:04<2:31:54, 14.02s/it] 

Вывод. Действительно, Transformer - мощная штука. Позволяет использовать уже обученные модели - дообучение позволяет создавать свои модели на основе уже предобученных моделей.